# Formatting and analysing a DEM

This is a brief tutorial that shows how to format and then plot a tiff that you downloaded from the internet. It serves as an alternative to looking at a DEM in a GIS. 

First, you need to import some of our tools.

In [2]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
from lsdviztools.lsdplottingtools import lsdmap_basicmanipulation as lsdm_bm
import lsdviztools.lsdmapwrappers as lsdmw

Now, point the tool to the data. Here we downloaded some data from the Scottish goverment lidar portal. You should download your own DEM and replace the filename. 

First we convert to UTM. 

In [ ]:
RasterFile = "NT65NE_50CM_DTM_PHASE3.tif"
DataDirectory = "/LSDTopoTools/data/lammermuir/"
gio.convert2UTM(DataDirectory, RasterFile,minimum_elevation=0.01,resolution=3)

Now convert the file to bil format (for LSDTopoTools)

In [ ]:
newRasterFile = "NT65NE_50CM_DTM_PHASE3_UTM.tif"
gio.convert2bil(DataDirectory, newRasterFile,minimum_elevation=0)

You can also convert for LSDTopoTools directly (this lumps the previous two steps together):

In [ ]:
RasterFile = "NT66SE_50CM_DTM_PHASE3.tif"
gio.convert4lsdtt(DataDirectory, RasterFile,minimum_elevation=0.01,resolution=3)

Now get some derivative rasters using lsdtopotools

In [5]:
lsdtt_parameters = {"write_hillshade" : "true",  
                    "surface_fitting_radius" : "7",
                    "print_curvature" : "true",
                    "print_dinf_drainage_area_raster" : "true"}
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = "NT66SE_50CM_DTM_PHASE3_UTM",
                                 write_prefix= "NT66SE_50CM_DTM_PHASE3_UTM",
                                 read_path = "/LSDTopoTools/data/lammermuir/",
                                 write_path = "/LSDTopoTools/data/lammermuir/",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()

The command line tool is: lsdtt-basic-metrics
The driver name is: Test_01
The read path is: /LSDTopoTools/data/lammermuir/
The write path is: /LSDTopoTools/data/lammermuir/
The read prefix is: NT66SE_50CM_DTM_PHASE3_UTM
The write prefix is: NT66SE_50CM_DTM_PHASE3_UTM
The parameter dictionary is:
{'write_hillshade': 'true', 'surface_fitting_radius': '7', 'print_curvature': 'true', 'print_dinf_drainage_area_raster': 'true'}


In [6]:
lsdtt_drive.run_lsdtt_command_line_tool()

Done writing the driver file
I've finised writing the driver file. Let me run LSDTT for you.
